In [2]:
import json

import requests
import pandas as pd

In [3]:
with open("../../secrets.json", 'r') as fr:
    content = "".join(fr.readlines())
    API_KEY = json.loads(content)['eod_histo_data_api_key']

### Get user data

In [4]:
url = f"https://eodhd.com/api/user?api_token={API_KEY}&fmt=json"
data = requests.get(url).json()
data

### Get Exchanges

In [3]:
exchanges = pd.read_json(f"https://eodhistoricaldata.com/api/exchanges-list/?api_token={API_KEY}")
# exchanges.head()

,Name,Code,OperatingMIC,Country,Currency,CountryISO2,CountryISO3
0,USA Stocks,US,"XNAS, XNYS",USA,USD,US,USA
1,London Exchange,LSE,XLON,UK,GBP,GB,GBR
2,Toronto Exchange,TO,XTSE,Canada,CAD,CA,CAN
3,NEO Exchange,NEO,NEOE,Canada,CAD,CA,CAN
4,TSX Venture Exchange,V,XTSX,Canada,CAD,CA,CAN
...,...,...,...,...,...,...,...
72,Money Market Virtual Exchange,MONEY,None,Unknown,Unknown,,
73,Europe Fund Virtual Exchange,EUFUND,None,Unknown,EUR,,
74,Istanbul Stock Exchange,IS,XIST,Turkey,TRY,TR,TUR
75,Cryptocurrencies,CC,CRYP,Unknown,Unknown,,


### Get Contracts

In [16]:
exchanges[:40]

,Name,Code,OperatingMIC,Country,Currency,CountryISO2,CountryISO3
0,USA Stocks,US,"XNAS, XNYS",USA,USD,US,USA
1,London Exchange,LSE,XLON,UK,GBP,GB,GBR
2,Toronto Exchange,TO,XTSE,Canada,CAD,CA,CAN
3,NEO Exchange,NEO,NEOE,Canada,CAD,CA,CAN
4,TSX Venture Exchange,V,XTSX,Canada,CAD,CA,CAN
5,Berlin Exchange,BE,XBER,Germany,EUR,DE,DEU
6,Hamburg Exchange,HM,XHAM,Germany,EUR,DE,DEU
7,XETRA Stock Exchange,XETRA,XETR,Germany,EUR,DE,DEU
8,Dusseldorf Exchange,DU,XDUS,Germany,EUR,DE,DEU
9,Hanover Exchange,HA,XHAN,Germany,EUR,DE,DEU


In [19]:
EXCHANGE_CODE = "US"
contracts = pd.read_json(f"https://eodhistoricaldata.com/api/exchange-symbol-list/{EXCHANGE_CODE}?api_token={API_KEY}&fmt=json")

,Code,Name,Country,Exchange,Currency,Type,Isin
0,0P0000A412,Franklin U.S. Opportunities Fund A,USA,PINK,USD,FUND,None
1,0P0000RX5G,Franklin Floating Rate Fund PLC W USD Dis,USA,PINK,USD,FUND,None
2,0P0001GXZ7,Oaktree Global Credit Fund P,USA,OTC,USD,FUND,None
3,0P0001NGF5,Liontrust GF Sust Fut MA Glbl B8 Acc USD,USA,PINK,USD,FUND,None
4,A,Agilent Technologies Inc,USA,NYSE,USD,Common Stock,US00846U1016
...,...,...,...,...,...,...,...
47683,ZYME,Zymeworks Inc. Common Stock,USA,NASDAQ,USD,Common Stock,CA98985W1023
47684,ZYXI,Zynex Inc,USA,NASDAQ,USD,Common Stock,US98986M1036
47685,ZZHGF,ZhongAn Online P & C Insurance Co. Ltd,USA,PINK,USD,Common Stock,None
47686,ZZHGY,ZhongAn Online P & C Insurance Co. Ltd,USA,PINK,USD,Common Stock,None


In [20]:
contracts['Type'].value_counts()

FUND               23888
Common Stock       18083
ETF                 3737
Mutual Fund         1329
Preferred Stock      508
Unit                 134
Note                   3
ETC                    2
Bond                   1
Name: Type, dtype: int64

In [32]:
# show contracts of type 'Stock'
contracts[contracts['Type'] == 'ETF'][:50]

,Code,Name,Country,Exchange,Currency,Type,Isin
6,AAA,Listed Funds Trust - AAF First Priority CLO Bo...,USA,NYSE ARCA,USD,ETF,US53656F6566
26,AAAU,Goldman Sachs Physical Gold ETF,USA,BATS,USD,ETF,US38150K1034
91,AADR,AdvisorShares Dorsey Wright ADR ETF,USA,NASDAQ,USD,ETF,US00768Y2063
187,AAPB,GraniteShares ETF Trust - GraniteShares 2x Lon...,USA,NASDAQ,USD,ETF,US38747R8842
188,AAPD,Direxion Shares ETF Trust - Direxion Daily AAP...,USA,NASDAQ,USD,ETF,US25461A3041
192,AAPR,Innovator Equity Defined Protection ETF - 2 Yr...,USA,BATS,USD,ETF,None
194,AAPU,Direxion Shares ETF Trust - Direxion Daily AAP...,USA,NASDAQ,USD,ETF,US25461A8743
195,AAPX,Etf Opportunities Trust - T Rex 2x Long Apple ...,USA,BATS,USD,ETF,None
196,AAPY,Kurv Yield Premium Strategy Apple (AAPL) ETF,USA,BATS,USD,ETF,None
226,AAXJ,iShares MSCI All Country Asia ex Japan ETF,USA,NASDAQ,USD,ETF,US4642881829


### Get Historical Quotes

In [110]:
FROM = "2022-02-23"
TO = "2022-03-23"
TICKER = "NVDA.US"
url = f"https://eodhistoricaldata.com/api/eod/{TICKER}?api_token={API_KEY}&from={FROM}&to={TO}"
quotes = pd.read_csv(url, skipfooter=1)
quotes.head()

#### Split/Dividend Adjustment

In [100]:
quotes['k'] = quotes['Close'] / quotes['Adjusted_close']
quotes['Open'] = quotes['Open'] / quotes['k']
quotes['High'] = quotes['High'] / quotes['k']
quotes['Low'] = quotes['Low'] / quotes['k']
quotes.drop(['Close', 'k'], axis='columns', inplace=True)
quotes.rename(columns={"Adjusted_close": "Close"}, inplace=True)

In [106]:
quotes.head()

,Date,Open,High,Low,Close,Volume
0,2022-02-23,237.929202,241.457756,222.924928,223.7846,56651078
1,2022-02-24,210.069827,237.909202,208.820303,237.3894,73580062
2,2022-02-25,237.119464,242.077571,233.720762,241.4778,52886551
3,2022-02-28,239.828400,246.555831,236.979589,243.7569,47731859
4,2022-03-01,242.817293,243.676965,231.231717,234.6804,41205602


### Get Dividends And Splits

In [88]:
url = f"https://eodhistoricaldata.com/api/div/{TICKER}?api_token={API_KEY}&from={FROM}&fmt=json"
dividends = requests.get(url=url)

In [89]:
dividends.json()

[{'date': '2017-05-19',
  'declarationDate': None,
  'recordDate': '2017-05-23',
  'paymentDate': '2017-06-14',
  'period': 'Quarterly',
  'value': 0.035,
  'unadjustedValue': 0.14,
  'currency': 'USD'},
 {'date': '2017-08-22',
  'declarationDate': '2017-08-10',
  'recordDate': '2017-08-24',
  'paymentDate': '2017-09-18',
  'period': 'Quarterly',
  'value': 0.035,
  'unadjustedValue': 0.14,
  'currency': 'USD'},
 {'date': '2017-11-22',
  'declarationDate': '2017-11-09',
  'recordDate': '2017-11-24',
  'paymentDate': '2017-12-15',
  'period': 'Quarterly',
  'value': 0.0375,
  'unadjustedValue': 0.15,
  'currency': 'USD'},
 {'date': '2018-02-22',
  'declarationDate': '2018-02-08',
  'recordDate': '2018-02-23',
  'paymentDate': '2018-03-16',
  'period': 'Quarterly',
  'value': 0.0375,
  'unadjustedValue': 0.15,
  'currency': 'USD'},
 {'date': '2018-05-23',
  'declarationDate': '2018-05-10',
  'recordDate': '2018-05-24',
  'paymentDate': '2018-06-15',
  'period': 'Quarterly',
  'value': 0.

In [90]:
url = f"https://eodhistoricaldata.com/api/splits/{TICKER}?api_token={API_KEY}&from={FROM}"
splits = requests.get(url=url)

In [91]:
splits.text

'Date,"Stock Splits"\n2021-07-20,4.000000/1.000000\n49'

### Speedtest historical quotes

In [104]:
for TICKER in ["NVDA.US", "TSLA.US", "MSFT.US", "AMZN.US", "NFLX.US"]:
    url = f"https://eodhistoricaldata.com/api/eod/{TICKER}?api_token={API_KEY}&from={FROM}&to={TO}"
    quotes = requests.get(url=url)
    # print("-", end="")
    # url = f"https://eodhistoricaldata.com/api/div/{TICKER}?api_token={API_KEY}&from={FROM}&fmt=json"
    # dividends = requests.get(url=url)
    # print("-", end="")
    # url = f"https://eodhistoricaldata.com/api/splits/{TICKER}?api_token={API_KEY}&from={FROM}"
    # splits = requests.get(url=url)
    print("-")


-
-
-
-
-
